In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.4 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import pandas as pd
import os
import json
import geopandas as gpd
import geemap
import time

# Initialize
ee.Authenticate()
ee.Initialize(project='unicef-ccri')

In [ ]:
import rasterio
import numpy as np

# Path to the GeoTIFF file
tif_path = "/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/MHI.tif"

# Calculate percentiles dynamically
with rasterio.open(tif_path) as src:
    data = src.read(1)
    data = data[data != src.nodata]  # remove nodata
    percentiles = [75, 80, 85, 90, 95]
    percentile_values = {f'_Percentile{p}': float(np.nanpercentile(data, p)) for p in percentiles}

print(percentile_values)


{'_Percentile75': 6.270778656005859, '_Percentile80': 6.507885456085205, '_Percentile85': 6.814867973327637, '_Percentile90': 7.164295196533203, '_Percentile95': 7.6205153465271}


In [ ]:

# Load assets
risk_index = ee.Image("projects/unicef-ccri/assets/MHI")
worldpop = ee.Image("projects/unicef-ccri/assets/worldpop_2024")
childpop = ee.ImageCollection("projects/unicef-ccri/assets/childpop_constrained_final_2024")
pop_target_res = childpop.first().projection().nominalScale()

# Mosaic child population
mosaic_pop = childpop.mosaic()


# Admin level and AOI feature collection
adm_level = 'adm0'
aois = ee.FeatureCollection(f"projects/unicef-ccri/assets/georepo_{adm_level}")

# Function to summarize population
def summarize_population(adm_level, th_name, th_value):
    exposed_pop = mosaic_pop.updateMask(risk_index.gt(th_value))
    aois = ee.FeatureCollection(f"projects/unicef-ccri/assets/georepo_{adm_level}")

    population_by_aoi = exposed_pop.reduceRegions(
        collection=aois,
        reducer=ee.Reducer.sum(),
        scale=pop_target_res.getInfo(),  # convert ee.Number to native for .reduceRegions
        crs='EPSG:4326'
    )

    def set_exposure(feature):
        return feature.set('child_population_exposed', feature.get('sum'))

    final_collection = population_by_aoi.map(set_exposure)

    task = ee.batch.Export.table.toDrive(
        collection=final_collection,
        description=f'Multi_Hazard_intensity_Exposure_{adm_level}_TH{th_name}',
        fileFormat='CSV',
        selectors=['ISO3', 'child_population_exposed', 'name'],
        folder='CCRI_results_misc'
    )
    task.start()

# Use computed thresholds from step 1
thresholds = percentile_values

#loop through threosholds
for th_name, th_value in thresholds.items():
    summarize_population(adm_level, th_name, th_value)


In [ ]:
df_exposed_75 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_Exposure_adm0_TH_Percentile75.csv')
df_exposed_80 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_Exposure_adm0_TH_Percentile80.csv')
df_exposed_85 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_Exposure_adm0_TH_Percentile85.csv')
df_exposed_90 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_Exposure_adm0_TH_Percentile90.csv')
df_exposed_95 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_Exposure_adm0_TH_Percentile95.csv')

In [ ]:
df_uregion = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/misc/UNICEF_PROG_REG_GLOBAL.csv')
df_uregion

,Regional_Grouping,Region,Region_Code,Country,ISO3Code
0,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Cambodia,KHM
1,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,China,CHN
2,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Cook Islands,COK
3,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Democratic People's Republic of Korea,PRK
4,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Fiji,FJI
...,...,...,...,...,...
153,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Nigeria,NGA
154,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Sao Tome and Principe,STP
155,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Senegal,SEN
156,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Sierra Leone,SLE


In [ ]:
df_iso3_sum_75 = df_exposed_75.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_80 = df_exposed_80.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_85 = df_exposed_85.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_90 = df_exposed_90.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_95 = df_exposed_95.groupby('ISO3')['child_population_exposed'].sum().reset_index()


In [ ]:
print(df_iso3_sum_75['child_population_exposed'].sum())
print(df_iso3_sum_80['child_population_exposed'].sum())
print(df_iso3_sum_85['child_population_exposed'].sum())
print(df_iso3_sum_90['child_population_exposed'].sum())
print(df_iso3_sum_95['child_population_exposed'].sum())

1576727644.8511305
1460900147.7358732
1294234805.1258335
1033541021.3762696
642479924.5431439


In [ ]:
# Merge the three DataFrames on 'ISO3'
df_merged = df_iso3_sum_75.merge(
    df_iso3_sum_80, on='ISO3', suffixes=('_75', '_80')
).merge(
    df_iso3_sum_90, on='ISO3'
).rename(columns={'child_population_exposed': 'child_population_exposed_90'})

# Merge with df_uregion to bring in 'Region'
df_final = df_merged.merge(
    df_uregion[['ISO3Code', 'Region']],
    left_on='ISO3',
    right_on='ISO3Code',
    how='left'
).drop(columns=['ISO3Code'])

# Print the final sums for verification
print(df_iso3_sum_75['child_population_exposed'].sum())
print(df_iso3_sum_80['child_population_exposed'].sum())
print(df_iso3_sum_90['child_population_exposed'].sum())


1576727644.8511305
1460900147.7358732
1033541021.3762696


In [ ]:
df_final.to_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/misc/exposure_by_iso3_TH_pixel.csv', index=False)

In [ ]:
# Group by Region and sum exposures
df_summary_by_region = df_final.groupby('Region')[[
    'child_population_exposed_75',
    'child_population_exposed_80',
    'child_population_exposed_90'
]].sum().reset_index()

# Display the summary
print(df_summary_by_region)


                            Region  child_population_exposed_75  \
0            East Asia and Pacific                 3.041776e+08   
1      Eastern and Southern Africa                 9.031715e+07   
2          Europe and Central Asia                 4.579931e+07   
3  Latin America and the Caribbean                 6.687929e+07   
4     Middle East and North Africa                 1.592019e+08   
5                       South Asia                 5.853015e+08   
6          West and Central Africa                 2.503194e+08   

   child_population_exposed_80  child_population_exposed_90  
0                 2.833968e+08                 2.043114e+08  
1                 7.203346e+07                 3.544693e+07  
2                 3.999622e+07                 2.455023e+07  
3                 5.686608e+07                 3.184085e+07  
4                 1.517549e+08                 1.206191e+08  
5                 5.676766e+08                 4.205843e+08  
6                 2.312517e+0

In [ ]:
for col in ['child_population_exposed_75', 'child_population_exposed_80', 'child_population_exposed_90']:
    df_summary_by_region[col] = (df_summary_by_region[col] / 1e6).round(2)

In [ ]:
# Compute totals for all numeric columns
totals = df_summary_by_region.drop(columns='Region').sum()

# Create a total row
df_total_row = pd.DataFrame([{'Region': 'Total', **totals.to_dict()}])

# Append to the summary DataFrame
df_summary_by_region = pd.concat([df_summary_by_region, df_total_row], ignore_index=True)


In [21]:
df_summary_by_region

,Region,child_population_exposed_75,child_population_exposed_80,child_population_exposed_90
0,East Asia and Pacific,304.18,283.40,204.31
1,Eastern and Southern Africa,90.32,72.03,35.45
2,Europe and Central Asia,45.80,40.00,24.55
3,Latin America and the Caribbean,66.88,56.87,31.84
4,Middle East and North Africa,159.20,151.75,120.62
5,South Asia,585.30,567.68,420.58
6,West and Central Africa,250.32,231.25,175.46
7,Total,1502.00,1402.98,1012.81
